# 🚀 YOLOv12-Face Enhanced - Training (Corrigé)

Entraînement du modèle YOLOv12-Face avec modules d'attention Enhanced
**Version corrigée utilisant le fork local d'Ultralytics**

## 1. 📦 Installation des dépendances

In [ ]:
# IMPORTANT: Ne PAS installer ultralytics via pip
# On utilise le fork local dans ./ultralytics/

# Installer seulement les dépendances (sans ultralytics)
!pip install -r requirements.txt -q

# Vérifier l'installation
import torch
print(f"✅ PyTorch: {torch.__version__}")
print(f"✅ CUDA disponible: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"✅ GPU: {torch.cuda.get_device_name(0)}")

## 2. 🔧 Configuration du fork local

In [ ]:
import os
import sys
from pathlib import Path

# Ajouter le répertoire courant au PYTHONPATH pour utiliser le fork local
current_dir = Path.cwd()
if str(current_dir) not in sys.path:
    sys.path.insert(0, str(current_dir))

print(f"📁 Répertoire de travail: {current_dir}")
print(f"📁 Python path ajouté: {current_dir}")

# Vérifier la structure du projet
ultralytics_dir = current_dir / 'ultralytics'
enhanced_file = ultralytics_dir / 'nn' / 'modules' / 'enhanced.py'

print(f"\n🔍 Vérification de la structure:")
print(f"   • ultralytics/: {'✅' if ultralytics_dir.exists() else '❌'}")
print(f"   • enhanced.py: {'✅' if enhanced_file.exists() else '❌'}")

if not enhanced_file.exists():
    print("❌ ERREUR: Fichier enhanced.py manquant!")
    print("💡 Assurez-vous d'être dans le bon répertoire avec le fork Ultralytics")

## 3. 🧪 Test des imports Enhanced

In [ ]:
# Test des imports étape par étape
print("🧪 Test des imports Enhanced...")

try:
    # Import du fork local
    print("1. Import ultralytics (fork local)...")
    from ultralytics import YOLO
    print("   ✅ YOLO importé")
    
    # Import des modules Enhanced
    print("2. Import modules Enhanced...")
    from ultralytics.nn.modules.enhanced import A2Module, RELAN
    print("   ✅ A2Module et RELAN importés")
    
    # Test rapide des modules
    print("3. Test des modules...")
    x = torch.randn(1, 64, 32, 32)
    
    a2 = A2Module(64, 64)
    out_a2 = a2(x)
    print(f"   ✅ A2Module: {x.shape} -> {out_a2.shape}")
    
    relan = RELAN(64, 64)
    out_relan = relan(x)
    print(f"   ✅ RELAN: {x.shape} -> {out_relan.shape}")
    
    print("\n🎉 TOUS LES IMPORTS FONCTIONNENT !")
    
except Exception as e:
    print(f"❌ ERREUR: {type(e).__name__}: {e}")
    import traceback
    traceback.print_exc()
    print("\n💡 Vérifiez que vous êtes dans le bon répertoire avec le fork Ultralytics")

## 4. 📥 Préparation du dataset WIDERFace

In [ ]:
# Télécharger et préparer le dataset WIDERFace

# Vérifier si le dataset existe déjà
widerface_config = Path('ultralytics/cfg/datasets/widerface.yaml')

if widerface_config.exists():
    print("✅ Configuration WIDERFace trouvée")
    
    # Lire la config pour voir le path des données
    import yaml
    with open(widerface_config, 'r') as f:
        config = yaml.safe_load(f)
    
    dataset_path = Path(config.get('path', 'datasets/widerface'))
    print(f"📁 Path dataset: {dataset_path}")
    
    if dataset_path.exists():
        print("✅ Dataset WIDERFace déjà présent")
    else:
        print("📥 Dataset non trouvé, préparation nécessaire...")
        if Path('scripts/prepare_widerface.py').exists():
            !python scripts/prepare_widerface.py
        else:
            print("⚠️ Script de préparation non trouvé. Téléchargement manuel nécessaire.")
else:
    print("❌ Configuration WIDERFace non trouvée")

## 5. 🏋️ Configuration de l'entraînement

In [ ]:
# Configuration de l'entraînement
MODEL_ENHANCED = 'ultralytics/cfg/models/v12/yolov12-face-enhanced.yaml'
MODEL_BASE = 'ultralytics/cfg/models/v12/yolov12-face.yaml'
DATA = 'ultralytics/cfg/datasets/widerface.yaml'

# Paramètres d'entraînement
EPOCHS = 50  # Réduire pour test rapide (10-20)
BATCH_SIZE = 16 if torch.cuda.is_available() else 4
IMGSZ = 640
DEVICE = 0 if torch.cuda.is_available() else 'cpu'

# Choisir le modèle
if Path(MODEL_ENHANCED).exists():
    MODEL = MODEL_ENHANCED
    print("🧠 Utilisation du modèle Enhanced avec A2Module et RELAN")
elif Path(MODEL_BASE).exists():
    MODEL = MODEL_BASE
    print("⚙️ Utilisation du modèle de base YOLOv12-Face")
else:
    print("❌ Aucun modèle trouvé, utilisation de yolov12n")
    MODEL = 'yolov12n'

print(f"\n⚙️ Configuration:")
print(f"   • Modèle: {MODEL}")
print(f"   • Données: {DATA}")
print(f"   • Epochs: {EPOCHS}")
print(f"   • Batch size: {BATCH_SIZE}")
print(f"   • Image size: {IMGSZ}")
print(f"   • Device: {DEVICE}")

## 6. 🚀 Entraînement

In [ ]:
# Créer et entraîner le modèle
try:
    print("🚀 Démarrage de l'entraînement...")
    
    # Créer le modèle YOLO
    model = YOLO(MODEL)
    print(f"✅ Modèle chargé: {MODEL}")
    
    # Lancer l'entraînement
    results = model.train(
        data=DATA,
        epochs=EPOCHS,
        batch=BATCH_SIZE,
        imgsz=IMGSZ,
        device=DEVICE,
        project='runs/train',
        name='yolov12-face-enhanced',
        exist_ok=True,
        save=True,
        plots=True
    )
    
    print("\n🎉 Entraînement terminé !")
    
except Exception as e:
    print(f"❌ ERREUR pendant l'entraînement: {e}")
    import traceback
    traceback.print_exc()

## 7. 📊 Validation du modèle

In [ ]:
# Valider le modèle entraîné
BEST_MODEL = 'runs/train/yolov12-face-enhanced/weights/best.pt'

if Path(BEST_MODEL).exists():
    print("📊 Validation du modèle...")
    
    try:
        # Charger le meilleur modèle
        best_model = YOLO(BEST_MODEL)
        
        # Lancer la validation
        val_results = best_model.val(
            data=DATA,
            device=DEVICE,
            plots=True
        )
        
        print("✅ Validation terminée")
        print(f"📊 mAP50: {val_results.box.map50:.4f}")
        print(f"📊 mAP50-95: {val_results.box.map:.4f}")
        
    except Exception as e:
        print(f"❌ Erreur pendant la validation: {e}")
else:
    print("❌ Modèle non trouvé. Entraînement nécessaire.")

## 8. 📦 Export du modèle

In [ ]:
# Exporter en différents formats
if Path(BEST_MODEL).exists():
    try:
        model = YOLO(BEST_MODEL)
        
        print("📦 Export ONNX...")
        model.export(format='onnx', simplify=True)
        
        print("📦 Export TorchScript...")
        model.export(format='torchscript')
        
        print("✅ Exports terminés!")
        
    except Exception as e:
        print(f"❌ Erreur pendant l'export: {e}")
else:
    print("❌ Modèle non trouvé")

## 9. 🖼️ Test sur une image

In [ ]:
# Tester sur une image d'exemple
if Path(BEST_MODEL).exists():
    try:
        model = YOLO(BEST_MODEL)
        
        # Chercher des images de test
        test_images = []
        
        # Images de validation du dataset
        val_images_dir = Path('datasets/widerface/images/val')
        if val_images_dir.exists():
            test_images.extend(list(val_images_dir.glob('*.jpg'))[:3])
        
        # Images d'exemple d'Ultralytics
        example_images = Path('ultralytics/assets')
        if example_images.exists():
            test_images.extend(list(example_images.glob('*.jpg')))
        
        if test_images:
            test_image = str(test_images[0])
            print(f"🖼️ Test sur: {test_image}")
            
            # Lancer la détection
            results = model.predict(
                source=test_image,
                save=True,
                project='test_results',
                name='demo',
                exist_ok=True,
                conf=0.5
            )
            
            print(f"✅ Détection terminée: {len(results[0].boxes)} visages détectés")
            print("📁 Résultats sauvegardés dans test_results/demo/")
        else:
            print("❌ Aucune image de test trouvée")
    
    except Exception as e:
        print(f"❌ Erreur pendant le test: {e}")
else:
    print("❌ Modèle non trouvé")

## 10. 📈 Visualiser les résultats

In [ ]:
# Afficher les courbes d'entraînement
from IPython.display import Image, display

results_path = Path('runs/train/yolov12-face-enhanced')

if results_path.exists():
    print("📊 Résultats d'entraînement:")
    
    # Afficher les graphiques s'ils existent
    plots = ['results.png', 'confusion_matrix.png', 'F1_curve.png', 'PR_curve.png']
    
    for plot in plots:
        plot_path = results_path / plot
        if plot_path.exists():
            try:
                print(f"\n📊 {plot}:")
                display(Image(filename=str(plot_path), width=800))
            except Exception as e:
                print(f"   ❌ Erreur affichage {plot}: {e}")
                print(f"   📁 Disponible: {plot_path}")
        else:
            print(f"❌ {plot} non trouvé")
else:
    print("❌ Dossier de résultats non trouvé")

## 11. 💡 Résumé et commandes utiles

In [ ]:
print("🎉 ENTRAÎNEMENT YOLOv12-FACE ENHANCED TERMINÉ !")
print("=" * 50)

# Vérification finale
if Path(BEST_MODEL).exists():
    print(f"✅ Modèle entraîné: {BEST_MODEL}")
    print(f"📊 Résultats: {results_path}")
    
    model_size = Path(BEST_MODEL).stat().st_size / (1024*1024)
    print(f"📦 Taille du modèle: {model_size:.1f} MB")
    
    if 'enhanced' in str(MODEL).lower():
        print("\n🧠 Modèle Enhanced utilisé !")
        print("   • A2Module: Area Attention Module")
        print("   • RELAN: Residual Efficient Layer Aggregation Network")
    else:
        print("\n⚙️ Modèle de base utilisé")
        
    print("\n🔧 Commandes utiles:")
    print(f"# Test webcam:")
    print(f"from ultralytics import YOLO")
    print(f"model = YOLO('{BEST_MODEL}')")
    print(f"model.predict(source=0, show=True)")
    
    print(f"\n# Test sur vidéo:")
    print(f"model.predict(source='video.mp4', save=True)")
    
    print(f"\n# Validation:")
    print(f"model.val(data='{DATA}')")
    
else:
    print("❌ Entraînement échoué ou modèle non sauvegardé")
    print("💡 Vérifiez les erreurs ci-dessus")

print("\n🚀 Le modèle YOLOv12-Face Enhanced est prêt !")
print("\n📧 Pour support: vérifiez la documentation du projet")